In [1]:
version = "REPLACE_PACKAGE_VERSION"

---
# Assignment 1: Text Retrieval and Evaluation (100 pts)

In this assignment, we will build a miniature text retrieval system and evaluate its efficacy using standard metrics. 

In [2]:
# Configure nltk

import nltk

nltk_data_path = "assets/nltk_data"
if nltk_data_path not in nltk.data.path:
    nltk.data.path.append(nltk_data_path)

## Question 1: Load the data set (30 pts)

We will use a pre-processed version of the classic "[Cranfield collections](http://ir.dcs.gla.ac.uk/resources/test_collections/cran/)" as our data set for implementing a text retrieval system. The Cranfield collections contain 1,400 short documents from the aerodynamics field, along with a set of 225 queries and the corresponding relevance judgements (that is, what documents are deemed relevant to what queries by human experts). In what follows, we will first create three functions for loading the three basic components (documents, queries and relevance judgements), respectively. 

### Question 1a:  Load the documents (10 pts)

Complete the function below to load all Cranfield documents. Each of the 1,400 documents is stored as a single text file under `assets/cranfield/cranfieldDocs`. While the general file-reading procedures certainly work here, if you examine a sample file you will probably notice that the format of the content bears close resemblance to that of an HTML/XML file, in that it contains multiple "fields" demarcated by pairs of "tags" like `<TEXT>` and `</TEXT>`. This allows the use of XML-parsing tools in Python, such as the built-in [`xml.etree.ElementTree`](https://docs.python.org/3/library/xml.etree.elementtree.html#module-xml.etree.ElementTree) or external packages like [`lxml`](https://lxml.de/), which may be more useful than reading the files line by line. 

Regardless of the tools you will be using, we only consider anything in between `<TEXT>` and `</TEXT>` as the "content" of a document. We actually also need the identifier in between `<DOCNO>` and `</DOCNO>` to uniquely identify each document, but it is just a serial number from 1 to 1400. You can either grab that number or just keep a counter as you parse the documents in order. Once you grab the text in between `<TEXT>` and `</TEXT>` for each document, perform the following processing steps:

* **Tokenise the text.** For no good reason, just use `word_tokenize` from the `nltk` library to perform a quick tokenisation. 


* **Remove stop words.** Words like "a", "for", "the" etc. appear in almost every document and therefore, do not help distinguish one document from another. We remove them to drastically reduce the size of the vocabulary we have to deal with. The `stopwords` utility from `nltk.corpus` can provide us with a list of stop words in English. 


* **Stem each word.** Words sharing a common stem usually have related meanings, for example, "compute", "computer" and "computation". As far as understanding the main idea of a document is concerned (so that it can be accurately retrieved given a relevant query), it is arguably not very useful to distinguish such words. So we can further shrink our vocabulary by keeping only the word stems. We will use `PorterStemmer` from `nltk.stem.porter` for this purpose. 

Now that each document is comprised of a sequence of tokens, we store all documents in a `dict` indexed by the `doc_id`s, similar to the following:

```
{
    'd1'   : ['experiment', 'studi', ..., 'configur', 'experi', '.'], 
    'd2'   : ['studi', 'high-spe', ..., 'steadi', 'flow', '.'],
    ..., 
    'd1400': ['report', 'extens', ..., 'graphic', 'form', '.']
}
```

where we associate each document with a `doc_id` created by prefixing the document number found in between `<DOCNO>` and `</DOCNO>` by the letter "d". Even though in general we shouldn't rely on the order of the keys in a `dict`, **in this case please make sure the `doc_id`s are ordered as shown above**. In Python 3.7 or higher, a `dict` preserves insertion order, so as long as you add each document to your `dict` in ascending order of the document numbers, you should be good. This obviates the need for sorting when we need all documents in a nested list in a later assignment. 


**This function should return a `dict` of length 1400, where each key is a `str` representing a `doc_id` and each value is a `list` that contains the tokens resulting from processing a document as described above.**

In [3]:
import os
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def load_cranfield_docs():
    """
    Load and process cranfield documents
    """
    import xml.etree.ElementTree as ET
    
    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()
    
    import xml.etree.ElementTree as ET
    
    path = 'assets/cranfield/cranfieldDocs'
#     counter = 1
    tmp_docs = {}
    
    for filename in os.listdir(path):
        fullname = os.path.join(path, filename)
        tree = ET.parse(fullname)
        root = tree.getroot()
        
        tmp = [words for words in word_tokenize(root[4].text) if words not in stop_words]
        tmp = [stemmer.stem(w) for w in tmp]
        
        count = root[0].text.strip()
        
        tmp_docs[int(count)] = tmp
        
#         counter += 1
        
    mykeys = list(tmp_docs.keys())
    mykeys.sort()
    
    docs = {i: tmp_docs[i] for i in mykeys}
    docs = {'d'+str(i): v for i, v in docs.items()}
    
    # YOUR CODE HERE
#     raise NotImplementedError()
    
    return docs

In [4]:
# Autograder tests

stu_docs = load_cranfield_docs()

# Some sanity checks
assert isinstance(stu_docs, dict), "Q1a: Your function should return a dictionary. "
assert len(stu_docs) == 1400, "Q1a: Your dictionary should contain 1,400 documents. "

for idx, doc_id in enumerate(stu_docs, 1):
    assert f"d{idx}" == doc_id, f"Q1a: {doc_id} was not inserted in the correct order. "

# Some hidden tests

### Question 1b: Load the queries (10 pts)

Next, let's create a function for loading the Cranfield queries. The queries are all stored in a single text file,  `assets/cranfield/cranfield.queries`. It is not hard to tell that each line contains a query number and a piece of query text, separated by the first whitespace. 

We will identify each query in a way similar to how we identify each document. Each query is assigned a `q_id` formed by prefixing the query number found on the beginning of each line with the letter "q". We will also need to perform the same tokenisation, stop words removal and word stemming on each piece of query text as we have done on the documents (why?). Finally, we will drop the **ending** period (".") that is a **stand-alone** token at the end of every query. This helps prevent a query from being "superficially matched" to a document purely on the basis of two matching periods. However, we shall **not** strip off ending periods that are part of a token (e.g., "15.4**.**") or periods that occur in the middle of a query. 

We store all the queries in a `dict` indexed by `q_id`s, similar to the following:
```
{
    'q1'  : ['similar', 'law', ..., 'speed', 'aircraft'],
    'q2'  : ['structur', 'aeroelast', ..., 'speed', 'aircraft'], 
    ...
    'q225': ['design', 'factor', ..., 'number', '5']
}
```

Again, please make sure the queries are inserted in ascending order of the query numbers as shown above. 

**This function should return a `dict` of length 225, where each key is a `str` representing a `q_id` and each value is a `list` that contains the tokens resulting from processing a query as described above.**

In [5]:
def load_cranfield_queries():
    """
    Load and process cranfield queries
    """
    
    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()
    
    queries = {}
    
    with open('assets/cranfield/cranfield.queries') as f:
        lines = f.readlines()
    
    for line in lines:
        sent = word_tokenize(line)
        key = sent[0]
        sent.pop(0)
        
        sent = [words for words in sent if words not in stop_words]
        sent = [stemmer.stem(w) for w in sent]
        
        if sent[-1] == '.':
            sent.pop(-1)
            
        
        queries['q'+key] = sent
    
    
    # YOUR CODE HERE
#     raise NotImplementedError()
    
    return queries

In [6]:
# Autograder tests

stu_queries = load_cranfield_queries()

# Some sanity checks
assert isinstance(stu_queries, dict), "Q1b: Your function should return a dictionary. "
assert len(stu_queries) == 225, "Q1b: Your dictionary should contain 225 queries. "

for idx, q_id in enumerate(stu_queries, 1):
    assert f"q{idx}" == q_id, f"Q1b: {q_id} was not inserted in the correct order. "

# Some hidden tests

del stu_queries

In [7]:
stu_queries = load_cranfield_queries()
stu_queries

{'q1': ['similar',
  'law',
  'must',
  'obey',
  'construct',
  'aeroelast',
  'model',
  'heat',
  'high',
  'speed',
  'aircraft'],
 'q2': ['structur',
  'aeroelast',
  'problem',
  'associ',
  'flight',
  'high',
  'speed',
  'aircraft'],
 'q3': ['problem', 'heat', 'conduct', 'composit', 'slab', 'solv', 'far'],
 'q4': ['criterion',
  'develop',
  'show',
  'empir',
  'valid',
  'flow',
  'solut',
  'chemic',
  'react',
  'ga',
  'mixtur',
  'base',
  'simplifi',
  'assumpt',
  'instantan',
  'local',
  'chemic',
  'equilibrium'],
 'q5': ['chemic',
  'kinet',
  'system',
  'applic',
  'hyperson',
  'aerodynam',
  'problem'],
 'q6': ['theoret',
  'experiment',
  'guid',
  'turbul',
  'couett',
  'flow',
  'behaviour'],
 'q7': ['possibl',
  'relat',
  'avail',
  'pressur',
  'distribut',
  'ogiv',
  'forebodi',
  'zero',
  'angl',
  'attack',
  'lower',
  'surfac',
  'pressur',
  'equival',
  'ogiv',
  'forebodi',
  'angl',
  'attack'],
 'q8': ['method',
  '-dash',
  'exact',
  'appro

### Question 1c: Load the relevance judgements (10 pts)

Finally, let's create a function for loading the relevance judgements from `assets/cranfield/cranfield.reljudge`. Each line of the file contains two whitespace-separated numbers representing a query number and a document number. For example, the first few lines read:

```
1 184
1 29
1 31
```

which, in our notations, can be interpreted as

```
d184 is relevant to q1
d29  is relevant to q1
d31  is relevant to q1
```

A query could have any number of relevant documents (or none). We store all the relevance judgements in a `dict` indexed by `q_id`s, similar to:
```
{
    'q1': ['d184', 'd29', ..., 'd879', 'd880'],
    'q2': ['d12', 'd15', ..., 'd864', 'd658'], 
    ...
    'q225': ['d1379', 'd1305', ..., 'd1075', 'd1213']
}
```

Likewise, please make sure the relevance judgements are inserted in ascending order of the query numbers as shown above. 


**This function should return a `dict` of length 225, where each key is a `str` representing a `q_id` and each value is a `list` that contains the `doc_id` of the relevant documents.**

In [8]:
from collections import defaultdict

def load_cranfield_reljudges():
    """
    Load and process cranfield relevance judgements
    """
    reljudges = {}
    
    counter = 1
    
    with open('assets/cranfield/cranfield.reljudge') as f:
        lines = f.readlines()

    tmp = []    
    for line in lines:
        tmp.append(line.split())

    d = defaultdict(list)

    for k, v in tmp:
        d['q'+k].append('d'+v)

    # YOUR CODE HERE
#     raise NotImplementedError()
    
    return dict(d)

In [9]:
# Autograder tests

stu_reljudges = load_cranfield_reljudges()

# Some sanity checks
assert isinstance(stu_reljudges, dict), "Q1c: Your function should return a dictionary. "
assert len(stu_reljudges) == 225, "Q1c: Your dictionary should contain relevance judgements for 225 queries. "

for idx, q_id in enumerate(stu_reljudges, 1):
    assert f"q{idx}" == q_id, f"Q1c: {q_id} was not inserted in the correct order. "

# Some hidden tests

del stu_reljudges

## Question 2: Build an inverted index (20 pts)

Now we can build an inverted index out of the Cranfield collection. Our inverted index should be a `dict` that looks similar to the following:

```
{
    'experiment': {'d1': [1, [0]], ..., 'd30': [2, [12, 40]], ..., 'd123': [3, [11, 45, 67]], ...}, 
    
    'studi': {'d1': [1, [1]], 'd2': [2, [0, 36]], ..., 'd207': [3, [19, 44, 59]], ...}
    
    ...
}
```

where the keys of the dictionary are the *terms*. Each term is associated with a `dict` indexed by the `doc_id` of the documents which contain that term. Along with each `doc_id` we also store the frequency and the positions of the term in each document, similar to what is shown in the lecture slides. 

More concretely, the above example shows two terms, "experiment" and "studi". In the dictionary associated with the term "experiment", the item `'d1': [1, [0]]` means 

* `d1` is a document that contains the term "experiment"; and 
* the term "experiment" appears `1` time in `d1` at index `0`. 

Similarly, in the dictionary associated with the term "studi", the item `'d207': [3, [19, 44, 59]]` means:

* `d207` is a document that contains the term "studi"; and 
* the term "studi" appears `3` times in `d207` at indices `19`, `44` and `59`. 


We also need to decide what tokens are considered as terms. Not all tokens are qualified as terms to be tracked in our inverted index, except for those "significant" ones that appear in a "reasonable" number of documents. If a token appears in only one document, it may well be just noise. However, we must also be cautious about setting too high a threshold to include enough terms for our retrieval task. 

Complete the function below to build such an inverted index. It accepts two arguments: `docs` is a collection of documents as we built in Q1a and `min_df` specifies the minimum number of documents a token must appear in (a.k.a document frequency, df) to be included in the inverted index as a term. For example, `min_df=1` means we include all tokens as terms, because every token must have appeared in at least one document. In other words, we only consider tokens whose document frequency is at least `min_df` as terms. 


**This function should return a `dict` representing the inverted index built out of `docs` as specified above, which is indexed by terms as determined by `min_df`.**

In [10]:
def build_inverted_index(docs, min_df=1):
    """
    Take a collection of documents and build an inverted index
    """
    d = defaultdict(list)
    
    inv_index = {}
    tmp = []
    pos = 0
    
    freq = ()
    for doc in docs:
        indicies = defaultdict(list)
        
        for i, item in enumerate(docs[doc]):
            indicies[item].append(i)
            
        uniques = list(dict.fromkeys(docs[doc]))
        
        for word in uniques:
#             indicies = defaultdict(list)
            frequency = docs[doc].count(word)


            if word not in inv_index:
                inv_index[word] = {}
                inv_index[word][doc] = [frequency,dict(indicies)[word]]
            else:

                inv_index[word][doc] = [frequency,dict(indicies)[word]]
#             inv_index[word].append({doc:[frequency,dict(indicies)[word]]})
            
    # YOUR CODE HERE
#     raise NotImplementedError()

    inv_index = {k: v for k,v in inv_index.items() if len(v) >= min_df}
        
    
    return dict(inv_index)

In [11]:
stu_inv_index = build_inverted_index(stu_docs, min_df=4)
stu_inv_index

{'experiment': {'d1': [1, [0]],
  'd11': [1, [46]],
  'd12': [1, [23]],
  'd16': [1, [62]],
  'd17': [1, [27]],
  'd19': [1, [24]],
  'd25': [1, [89]],
  'd29': [1, [129]],
  'd30': [2, [12, 40]],
  'd35': [1, [63]],
  'd37': [1, [80]],
  'd41': [1, [23]],
  'd43': [1, [0]],
  'd47': [1, [120]],
  'd53': [1, [0]],
  'd58': [1, [41]],
  'd69': [1, [68]],
  'd70': [1, [1]],
  'd78': [2, [80, 101]],
  'd84': [1, [3]],
  'd99': [2, [103, 146]],
  'd101': [1, [206]],
  'd103': [1, [56]],
  'd112': [1, [78]],
  'd115': [1, [108]],
  'd121': [1, [82]],
  'd123': [3, [11, 45, 67]],
  'd131': [1, [146]],
  'd137': [1, [3]],
  'd140': [1, [107]],
  'd142': [1, [16]],
  'd154': [1, [30]],
  'd156': [1, [2]],
  'd167': [1, [61]],
  'd168': [1, [94]],
  'd170': [1, [137]],
  'd171': [2, [8, 55]],
  'd173': [2, [2, 48]],
  'd176': [1, [3]],
  'd179': [2, [122, 133]],
  'd183': [1, [62]],
  'd184': [1, [54]],
  'd186': [3, [30, 46, 54]],
  'd187': [1, [0]],
  'd188': [1, [116]],
  'd191': [1, [122]],

In [12]:
# Autograder tests

stu_inv_index = build_inverted_index(stu_docs, min_df=1)

# Some sanity checks
assert isinstance(stu_inv_index, dict), "Q2: Your function should return a dictionary. "
assert stu_inv_index, "Q2: Your inverted index is empty. "

# All tokens should be included as terms when min_df=1
all_tokens = {t for tokens in stu_docs.values() for t in tokens}
all_terms = set(stu_inv_index.keys())

assert all_tokens == all_terms, "Q2: When min_df = 1, all tokens should be included as terms. "

# Some hidden tests

del stu_inv_index, all_tokens, all_terms

## Question 3: Retrieve and rank documents (20 pts)

Now let's write a function for retrieving and ranking documents. We will just follow the simple heuristic introduced in the lecture example. That is, for each *term* in a given query (remember, not all tokens are terms. How do you tell whether a token is a term or not, when given an inverted index?), we accumulate the term frequencies for each document that contains the term. Having repeated this procedure for all terms, we rank the documents found in descending order of their total term frequencies. If two documents tie for their total term frequencies, the document with a *lower* document number should be ranked higher; for example, if `d12` and `d100` have the same total term frequences for a given query, `d12` should be ranked higher than `d100` because `12 < 100`. Refer to the lecture slides titled "Ranking Documents: Example" in the "Inverted Index" slide deck for more details, where we have a short query "data science". 

Complete the function below to implement such a heuristic. The function takes as inputs an `inverted_index` as returned by the function in Q2, and some `queries` as returned by the function in Q1b. It also accepts an argument,  `max_docs`, to limit the number of documents retrieved to be at most `max_docs`. A special case is when `max_docs=-1`, where no upper limits on the number of documents retrieved should be imposed (i.e., just return all documents matched). The function should output a `dict` that contains an ordered `list` of retrieved documents for each query, similar to:

```
{
    'q1'  : ['d51', 'd874', ..., 'd717'], 
    'q2'  : ['d51', 'd1147', ..., 'd14'],
    ...,
    'q225': ['d1313', 'd996', ..., 'd193']
}
```

**This function should return a `dict` of length `len(queries)`, where each key is a `str` representing a `q_id` and each value is a `list` that contains the `doc_id` of the retrieved documents in order. `max_docs` determines the maximum number of documents to retrieve for each query.**

In [13]:
def retrieve_n_rank_docs(inverted_index, queries, max_docs=-1):
    """
    Retrieve documents in order of relevance from an inverted index based on some queries
    """
    import re
    ret_docs = {}
                 
    
    for query in queries:
        posting = []
        doc_list = []
        for word in queries[query]:
            if word in inverted_index:
                posting_dict = {}
                for i in range(len(inverted_index[word])):
                    key = list(inverted_index[word].items())[i][0]
                    val = list(inverted_index[word].items())[i][1][0]

                    posting_dict[key] = val
                    doc_list.append(key)
                posting.append({word:posting_dict})


        totals = {}
    
        for doc in set(doc_list):
            tot = 0
            for post in range(len(posting)):
                for q_term in posting[post]:
                    if doc in posting[post][q_term]:
                        tot += posting[post][q_term][doc]
                totals[doc] = tot

        totals = dict(sorted(totals.items(), key = lambda x: (-x[1], int(re.search(r'\d+',x[0]).group()))))
        
        if max_docs == -1:
            total_list = list(totals.keys())
        else:
            total_list = list(totals.keys())[:max_docs]
        
        
        ret_docs[query] = total_list

                
    
    # YOUR CODE HERE
#     raise NotImplementedError()
    
    return ret_docs


In [14]:
# Autograder tests

min_df = 10 # min_df won't change in the hidden tests
stu_inv_index = build_inverted_index(stu_docs, min_df=min_df)
stu_queries = load_cranfield_queries()

max_docs = 10 # max_docs may vary in the hidden tests
stu_ret_docs = retrieve_n_rank_docs(stu_inv_index, stu_queries, max_docs=max_docs)

# Some sanity checks
assert isinstance(stu_ret_docs, dict), "Q3: Your function should return a dictionary. "
assert len(stu_ret_docs) == len(stu_queries), "Q3: Your dictionary should have the same length as there are queries. "

for q_id in stu_ret_docs:
    
    assert q_id in stu_queries, f"Q3: When max_docs = {max_docs}, '{q_id}' in your dictionary is not a valid q_id. "
    
    assert len(stu_ret_docs[q_id]) <= max_docs, f"Q3: When max_docs = {max_docs}, your # retrieved docs ({len(stu_ret_docs[q_id])}) for {q_id} is bigger than max_docs. "

# Some hidden tests

del stu_inv_index, stu_queries, stu_ret_docs, min_df, max_docs

## Question 4: Evaluate your retrieval system (30 pts)

Is your mini retrieval system doing a good job at all? Now let's evaluate it against some common metrics covered in class. 

### Question 4a: Calculate precision and recall @ $n$ (10 pts)

Complete the function below for calculating precision and recall @ $n$. It takes as inputs a set of retrieved documents, `ret_docs`, as returned by the function in Q3, and a set of relevance judgements, `reljudges`, as returned by the function in Q1c, plus, of course, the argument `n`. **A special case arises when `n=-1` or when `n` exceeds the number of documents retrieved, where we calculate precision and recall based on all documents retrieved.** For example, if 10 documents were retrieved for a query, then `precision @ 12` is essentially the same as `precision @ 10` (and `precision @ -1` by our rules). The function should output two `dict`s containing the precision and recall @ `n` for each query in `ret_docs`, respectively, both similar to:

```
{
    'q1'  : 0.1,
    'q2'  : 0.3,
    ...,
    'q225': 0.2
}
```

**This function should return two `dict`s both of length `len(ret_docs)`, where each key is a `str` representing a `q_id` and each value is a `float` representing either precision or recall @ `n`.**

In [15]:
def calc_pre_rec_at_n(ret_docs, reljudges, n=-1):
    """
    Calculate precision and recall at n for each query in ret_docs
    """
    
    pre_at_n, rec_at_n = {}, {}

    
    # YOUR CODE HERE
#     raise NotImplementedError()
    for answer in reljudges:
        if (n == -1) | (n > len(ret_docs[answer])):
            rel_ret = 0
            rel = len(reljudges[answer])
            ret = len(ret_docs[answer])
            for found in ret_docs[answer]:
                if found in reljudges[answer]:
                    rel_ret += 1

            pre = rel_ret / ret
            rec = rel_ret / rel

            pre_at_n[answer] = pre

            rec_at_n[answer] = rec
        else:
            rel_ret = 0
            rel = len(reljudges[answer])
            ret = len(ret_docs[answer][:n])
            for found in ret_docs[answer][:n]:
                if found in reljudges[answer]:
                    rel_ret += 1

            pre = rel_ret / ret
            rec = rel_ret / rel

            pre_at_n[answer] = pre

            rec_at_n[answer] = rec
            
        

    
    return pre_at_n, rec_at_n

In [16]:
# Autograder tests
import math

min_df = 10 # min_df won't change in the hidden tests 
stu_inv_index = build_inverted_index(stu_docs, min_df=min_df)
stu_queries = load_cranfield_queries()
stu_reljudges = load_cranfield_reljudges()

max_docs = 10 # max_docs may vary in the hidden tests
stu_ret_docs = retrieve_n_rank_docs(stu_inv_index, stu_queries, max_docs=max_docs)

stu_pre_at_n, stu_rec_at_n = calc_pre_rec_at_n(stu_ret_docs, stu_reljudges, n=-1)

# Some sanity checks
assert isinstance(stu_pre_at_n, dict), "Q4a: Your function should return a dictionary for pre_at_n. "
assert isinstance(stu_rec_at_n, dict), "Q4a: Your function should return a dictionary for rec_at_n. "
assert len(stu_pre_at_n) == len(stu_ret_docs), "Q4a: Your pre_at_n should have the same length as there are queries. "
assert len(stu_rec_at_n) == len(stu_ret_docs), "Q4a: Your rec_at_n should have the same length as there are queries. "
assert stu_pre_at_n.keys() == stu_ret_docs.keys(), "Q4a: Your pre_at_n contains invalid queries. "
assert stu_rec_at_n.keys() == stu_ret_docs.keys(), "Q4a: Your rec_at_n contains invalid queries. "

# Precision and recall must be between 0 and 1
for q_id in stu_ret_docs:
    assert 0.0 <= stu_pre_at_n[q_id] <= 1.0, f"Q4a: Your precision @ n for {q_id} should be between 0 and 1. "
    assert 0.0 <= stu_rec_at_n[q_id] <= 1.0, f"Q4a: Your recall @ n for {q_id} should be between 0 and 1. "

# Check it is still correct even if n exceeds max_docs
stu_pre_at_n_max, stu_rec_at_n_max = calc_pre_rec_at_n(stu_ret_docs, stu_reljudges, n=(max_docs + 2))

assert all(map(math.isclose, stu_pre_at_n.values(), stu_pre_at_n_max.values())), "Q4a: When n exceeds max_docs, your function should return the same pre_at_n as when n=-1. "
assert all(map(math.isclose, stu_rec_at_n.values(), stu_rec_at_n_max.values())), "Q4a: When n exceeds max_docs, your function should return the same rec_at_n as when n=-1. "

# Some hidden tests

del stu_inv_index, stu_queries, stu_reljudges, stu_ret_docs
del stu_pre_at_n, stu_rec_at_n, stu_pre_at_n_max, stu_rec_at_n_max
del min_df, max_docs

### Question 4b: Calculate (mean) average precision (10 pts)

Another commonly used metric is (mean) average precision. Complete the function below to calculate the average precision for each query and the mean average precision for all queries. It accepts exactly the same arguments as the `calc_pre_rec_at_n` function does in Q4a, except that the argument `n` is now renamed to `cutoff`. But otherwise `cutoff` serves a similar purpose to that of `n`: **only documents ranked higher than or at `cutoff` are considered as "retrieved"**. Similarly, a special case arises when `cutoff=-1` or when `cutoff` exceeds the number of documents retrieved, where we consider all documents as retrieved. The function should output a `dict` representing the average precisions for each query and a `float` representing the mean average precision. The `dict` should look similar to:

```
{
    'q1'  : 0.03571428571428571,
    'q2'  : 0.08194444444444444,
    ...,
    'q225': 0.02023809523809524
}
```

**This function should return a `dict` of length `len(ret_docs)`, where each key is a `str` representing a `q_id` and each value is a `float` representing the average precision, and a `float` representing the mean average precision.**

In [17]:
def calc_avg_pre(ret_docs, reljudges, cutoff=-1):
    """
    Calculate (mean) average precision for each query in ret_docs
    """
    
    avg_pre, mean_avg_pre = {}, None
    
    # YOUR CODE HERE
#     raise NotImplementedError()

    for answer in reljudges:
        if (cutoff == -1) | (cutoff > len(ret_docs[answer])):
#             rel_ret = 0
            rel = len(reljudges[answer])
#             ret = 1
            
            running_pre = []
            rel_ret = 0
            ret = 1
            for found in ret_docs[answer]:

                if found in reljudges[answer]:
                    rel_ret += 1

                    pre = rel_ret / ret
                    
                    running_pre.append(pre)
                    
                    ret += 1
                else:
                    pre = rel_ret / ret
                    
                    ret +=1
            
            avg_pre[answer] = sum(running_pre)/rel


        else:
            
            rel = len(reljudges[answer])

            
            running_pre = []
            rel_ret = 0
            ret = 1
            for found in ret_docs[answer][:cutoff]:

                if found in reljudges[answer]:
                    rel_ret += 1

                    pre = rel_ret / ret
                    
                    running_pre.append(pre)
                    
                    ret += 1
                else:
                    pre = rel_ret / ret
                    
                    ret +=1
            
            avg_pre[answer] = sum(running_pre)/rel
            

    mean_avg_pre = sum(list(avg_pre.values()))/len(list(avg_pre.values()))

    
    
    return avg_pre, mean_avg_pre

In [18]:
# Autograder tests
import math

min_df = 10 # min_df won't change in the hidden tests 
stu_inv_index = build_inverted_index(stu_docs, min_df=min_df)
stu_queries = load_cranfield_queries()
stu_reljudges = load_cranfield_reljudges()

max_docs = 10 # max_docs may vary in the hidden tests
stu_ret_docs = retrieve_n_rank_docs(stu_inv_index, stu_queries, max_docs=max_docs)

stu_avg_pre, stu_mean_avg_pre = calc_avg_pre(stu_ret_docs, stu_reljudges, cutoff=-1)

# Some sanity checks
assert isinstance(stu_avg_pre, dict), "Q4b: Your function should return a dictionary for avg_pre. "
assert isinstance(stu_mean_avg_pre, float), "Q4b: Your function should return a float for mean_avg_pre. "
assert len(stu_avg_pre) == len(stu_ret_docs), "Q4b: Your avg_pre should have the same length as there are queries. "
assert stu_avg_pre.keys() == stu_ret_docs.keys(), "Q4b: Your avg_pre contains invalid queries. "

# Average precision must be between 0 and 1
for q_id in stu_avg_pre:
    assert 0.0 <= stu_avg_pre[q_id] <= 1.0, f"Q4b: Your average precision for {q_id} should be between 0 and 1. "

# Check it is still correct even if cutoff exceeds max_docs
stu_avg_pre_max, stu_mean_avg_pre_max = calc_avg_pre(stu_ret_docs, stu_reljudges, cutoff=(max_docs + 2))

assert all(map(math.isclose, stu_avg_pre.values(), stu_avg_pre_max.values())), "Q4b: When cutoff exceeds max_docs, your function should return the same results as when cutoff=-1. "

# Some hidden tests

del stu_inv_index, stu_queries, stu_reljudges, stu_ret_docs
del stu_avg_pre, stu_mean_avg_pre, stu_avg_pre_max, stu_mean_avg_pre_max
del min_df, max_docs

### Question 4c: Calculate (normalised) discounted cumulative gain (DCG) (10 pts)

Finally, let's try to evaluate our text retrieval system against normalised DCG (NDCG). The Cranfield collection by itself doesn't actually allow us to calculate NDCG, because NDCG requires a numerical relevance score for each document rather than just a binary relevance judgement as provided in the Cranfield collection. We circumvent this limitation by making the following assumptions.

For a given query, 

* any documents not in `reljudges` (i.e., those irrelevant documents) have a relevance score of 1; and

* relevant documents are assigned decreasing relevance scores in the order they appear in `reljudges`. For example, consider the following relevance judgements:

```
{
    'q1': ['d184', 'd29', 'd879', 'd880'],
    'q2': ['d12', 'd15', 'd658'] 
}
```
where we assign the following relevance scores to the relevant documents of each query:
```
{
    'q1': [5, 4, 3, 2],
    'q2': [4, 3, 2] 
}
```
In other words, the relevant document `d184` of `q1` is assigned a relevance score of 5, and the relevant document `d658` of `q2` is assigned a relevance score of 2. The *last* relevant document always gets a relevance score of 2, the *second last* relevant document always gets a relevance score of 3, and so on, until the first relevant document. 
The relevance scores increase by 1 each time we move away from the last relevant document, starting at 2.

Complete the function below for calculating NDCG under the above assumptions. It accepts the usual arguments such as `ret_docs`, `reljudges`, and `n`. Not surprisingly, when `n=-1` or when `n` exceeds the number of documents retrieved, we calculate NDCG based on all documents retrieved. The additional argument `base` specifies the base in the logarithm function we use to discount relevance scores. The function outputs a `dict` containing the NDCG of each query, similar to:
```
{
    'q1': 0.21628669853396418,
    'q2': 0.3984097639816684,
    ...,
    'q225': 0.1361909750034715
}
```

An important corner case arises when `n` exceeds the number of *relevant* documents. What should the ideal ranking be in that case? The top few documents in the ideal ranking are certainly the relevant documents in descending order of their relevant scores. But what documents should follow after the list of relevant documents is exhausted? What should their relevance scores be? 

**This function should return a `dict` of length `len(ret_docs)`, where each key is a `str` representing a `q_id` and each value is a `float` representing the NDCG.**

In [19]:
import math

def calc_NDCG_at_n(ret_docs, reljudges, n=-1, base=2):
    """
    Calculate NDCG at n for each query in ret_docs
    """
    
    ndcg = {}
    
# #     for ret in ret_docs:
# #         new_list = []
# #         for x in ret_docs[ret]:
# #             if x in reljudges[ret]:
# #                 new_list.append
# #         ret_docs[ret] = new_list
# # #         [x for x in ret_docs[ret] if x in reljudges[ret]]
                
                
#     for result in reljudges:
# #         ideal_rank = []

#         list_len = len(reljudges[result])
#         ideal_rank = list(range(2,list_len+2))
# #         print(ideal_rank)


#         list_len2 = len(ret_docs[result])
#         system_rank = list(range(2,list_len2+2))
# #         print(system_rank)
        
#         if (n == -1) | (n > list_len2):
#             longest_len = max(list_len,list_len2)
#             ideal_rank = ideal_rank[::-1]
#             system_rank = system_rank[::-1]
            
#         else:
#             longest_len = max(list_len,list_len2)
            
#             ideal_rank = ideal_rank[:n]
#             system_rank = system_rank[:n]
#             print('got this far')
#             print(ideal_rank)
            
#             ideal_rank = ideal_rank[::-1]
#             system_rank = system_rank[::-1]
            
#         if len(ideal_rank) < len(system_rank):
#             irrel = [1] * (len(system_rank)-len(ideal_rank))
#             ideal_rank.extend(irrel)
# #             print('check1')
#         else:
#             irrel = [1] * (len(ideal_rank)-len(system_rank))
#             system_rank.extend(irrel)
# #             print('check2')
                       
#         if base > longest_len:
#             #calculate discounted cum rank with no base
#             dcg = system_rank
#             i_dcg = ideal_rank
# #             print('check3')
       
#         else:
#             dcg = []
#             i_dcg = []
# #             print('check4')
# #             print(system_rank)
# #             print(ideal_rank)
#             counter = 1
#             for idx, ranking in enumerate(system_rank):
#                 if (base - idx) > 1:
#                     dcg.append(ranking)
#                     counter += 1
#                 else:
#                     dcg.append(0 if math.log(ranking,base) == 0 else ranking / math.log(counter, base))
#                     counter += 1
# #             print('check5')
                    
#             for idx, ranking in enumerate(ideal_rank):
#                 if (base - idx) > 1:
#                     i_dcg.append(ranking)
#                     counter += 1
#                 else:
#                     i_dcg.append(0 if math.log(ranking,base) == 0 else ranking / math.log(counter, base))
#                     counter += 1
#         print('--')
#         print(reljudges[result])
#         print(ret_docs[result])
#         print('---')
#         print(system_rank) 
#         print(ideal_rank)
#         print(' ')
#         print('dcg')            
#         print(dcg)
#         print('')
#         print('i_dcg')
#         print(i_dcg)
#         ndcg[result]= sum(dcg)/sum(i_dcg)


    ranking_dict = {}
    
    for rel in reljudges:
#         if (n == -1) | (n > len(reljudges[rel])):
        tmp_dict = {}
        counter = 1
        documents = reljudges[rel]
        for doc in documents:
            tmp_dict[doc] = (len(documents) + counter)
#                 print('got here')
#                 print(counter)
            counter = counter - 1
        #here
        ranking_dict[rel] = tmp_dict
        
#         else:
#             tmp_dict = {}
#             counter = 1
#             documents = reljudges[rel][:n]
#             for doc in documents:
#                 tmp_dict[doc] = (len(documents) + counter)
#                 counter = counter - 1
                
#             ranking_dict[rel] = tmp_dict
            
#     print(ranking_dict)        
    system_dict = {}
    for ret in ret_docs:
        
        if (n == -1) | (n > len(ret_docs[ret])):
            tmp_dict = {}
            documents = ret_docs[ret]
            for doc in documents:
                if doc in list(ranking_dict[ret].keys()): 
                    tmp_dict[doc] = ranking_dict[ret][doc]
                else:
                    tmp_dict[doc] = 1
            system_dict[ret] = tmp_dict
            
        else:
            tmp_dict = {}
            documents = ret_docs[ret][:n]
            for doc in documents:
                if doc in list(ranking_dict[ret].keys()): 
                    tmp_dict[doc] = ranking_dict[ret][doc]
                else:
                    tmp_dict[doc] = 1
            system_dict[ret] = tmp_dict
            
            
            
    print(system_dict)        
    for query in ranking_dict:
        ideal_list = list(ranking_dict[query].values())
        system_list = list(system_dict[query].values())
        
        print('before exted')
        print(ideal_list)
        print(system_list)
        
        if len(ideal_list) < len(system_list):
            irrel = [1] * (len(system_list)-len(ideal_list))
            ideal_list.extend(irrel)
#             print('check1')
        if len(system_list) < len(ideal_list):
            irrel = [1] * (len(ideal_list)-len(system_list))
            system_list.extend(irrel)
#             print('check2')

        print('after extend')
        print(ideal_list)
        print(system_list)
        
        if (n == -1) | (n > len(ideal_list)):
            ideal_list = ideal_list
            system_list = system_list
        else:
            ideal_list = ideal_list[:n]
            system_list = system_list[:n]
 
        if base > len(ideal_list):
            dcg = []
            i_dcg = []
            #calculate discounted cum rank with no base
            dcg = system_list
            i_dcg = ideal_list
#             print('check3')
       
        else:
            dcg = []
            i_dcg = []
#             print('check4')
#             print(system_rank)
#             print(ideal_rank)
            counter = 1
            for idx, ranking in enumerate(system_list):
#                 if ranking == 1:
#                     dcg.append(1)

                if (idx - 1) < base:
                    dcg.append(ranking)
                    counter += 1
                else:
                    dcg.append(0 if math.log(idx + 1,base) == 0 else ranking / math.log(idx - 1, base))
                    counter += 1
#             print('check5')
                    
            for idx, ranking in enumerate(ideal_list):
#                 if ranking == 1:
#                     i_dcg.append(1)
#                 else:
                if (idx - 1)  < base:
                    i_dcg.append(ranking)
                    counter += 1
                else:
                    i_dcg.append(0 if math.log(idx + 1,base) == 0 else ranking / math.log(idx -1, base))
                    counter += 1

        print(query)
        print('dcg')
        print(dcg)

        print('i_dcg')
        print(i_dcg)
        
        print(sum(dcg)/sum(i_dcg))
        
        ndcg[query] = sum(dcg)/sum(i_dcg)    

        

        
        
                
        
        
            
            
        
    
#     
    # YOUR CODE HERE
#     raise NotImplementedError()
    
    return ndcg

In [20]:
# Autograder tests
import math

min_df = 10 # min_df won't change in the hidden tests 
stu_inv_index = build_inverted_index(stu_docs, min_df=min_df)
stu_queries = load_cranfield_queries()
stu_reljudges = load_cranfield_reljudges()

max_docs = 10 # max_docs may vary in the hidden tests
stu_ret_docs = retrieve_n_rank_docs(stu_inv_index, stu_queries, max_docs=max_docs)

base = 2 # base may vary in the hidden tests
stu_ndcg = calc_NDCG_at_n(stu_ret_docs, stu_reljudges, n=-1, base=base)

{'q1': {'d51': 25, 'd874': 1, 'd486': 1, 'd329': 1, 'd114': 1, 'd1268': 1, 'd1328': 1, 'd156': 1, 'd522': 1, 'd717': 1}, 'q2': {'d51': 21, 'd1147': 1, 'd12': 25, 'd100': 1, 'd114': 1, 'd640': 1, 'd792': 1, 'd329': 1, 'd1169': 1, 'd14': 18}, 'q3': {'d707': 1, 'd144': 4, 'd542': 1, 'd329': 1, 'd395': 1, 'd730': 1, 'd158': 1, 'd77': 1, 'd486': 1, 'd623': 1}, 'q4': {'d189': 1, 'd575': 1, 'd179': 1, 'd1182': 1, 'd160': 1, 'd188': 1, 'd660': 1, 'd173': 1, 'd185': 1, 'd193': 1}, 'q5': {'d730': 1, 'd329': 1, 'd1066': 1, 'd14': 1, 'd163': 1, 'd401': 4, 'd753': 1, 'd44': 1, 'd92': 1, 'd368': 1}, 'q6': {'d798': 1, 'd97': 1, 'd99': 5, 'd927': 1, 'd1195': 1, 'd131': 1, 'd315': 1, 'd151': 1, 'd193': 1, 'd257': 3}, 'q7': {'d189': 1, 'd1347': 1, 'd423': 1, 'd174': 1, 'd1231': 1, 'd197': 1, 'd721': 1, 'd1307': 1, 'd1352': 1, 'd717': 1}, 'q8': {'d234': 1, 'd122': 11, 'd189': 1, 'd160': 1, 'd197': 5, 'd193': 1, 'd1040': 1, 'd1231': 1, 'd423': 1, 'd1248': 1}, 'q9': {'d270': 1, 'd45': 1, 'd97': 1, 'd123': 

In [21]:
stu_ndcg

# 26/ (math.log(4,3))

{'q1': 0.15952187061023484,
 'q2': 0.34027762791910704,
 'q3': 0.25423420771380834,
 'q4': 0.6887320904474082,
 'q5': 0.48912127754157025,
 'q6': 0.6794486447319658,
 'q7': 0.31209843789970354,
 'q8': 0.3351113934875358,
 'q9': 0.5252413221051111,
 'q10': 0.33336915808564477,
 'q11': 0.26158706278886346,
 'q12': 0.31209843789970354,
 'q13': 0.3989662088724355,
 'q14': 0.896244030149136,
 'q15': 0.896244030149136,
 'q16': 0.6274750387607337,
 'q17': 0.6887320904474082,
 'q18': 0.6834942147367407,
 'q19': 0.1501314273582182,
 'q20': 0.1501314273582182,
 'q21': 0.3989662088724355,
 'q22': 0.8690756680778096,
 'q23': 0.17749152326405668,
 'q24': 0.5933971332088596,
 'q25': 0.3157660215619981,
 'q26': 0.28931971178126076,
 'q27': 0.5252413221051111,
 'q28': 0.6887320904474082,
 'q29': 0.1501314273582182,
 'q30': 0.20768758704199872,
 'q31': 0.8690756680778096,
 'q32': 0.25144072180437854,
 'q33': 0.5816121398411317,
 'q34': 0.6739675506776865,
 'q35': 0.6043677684209259,
 'q36': 0.725690712

In [22]:
for x in stu_ndcg[0]:
    print(stu_ndcg[0][x])

KeyError: 0

In [ ]:
# Autograder tests
import math

min_df = 10 # min_df won't change in the hidden tests 
stu_inv_index = build_inverted_index(stu_docs, min_df=min_df)
stu_queries = load_cranfield_queries()
stu_reljudges = load_cranfield_reljudges()

max_docs = 10 # max_docs may vary in the hidden tests
stu_ret_docs = retrieve_n_rank_docs(stu_inv_index, stu_queries, max_docs=max_docs)

base = 2 # base may vary in the hidden tests
stu_ndcg = calc_NDCG_at_n(stu_ret_docs, stu_reljudges, n=-1, base=base)

# Some sanity checks
assert isinstance(stu_ndcg, dict), "Q4c: Your function should return a dictionary. "
assert len(stu_ndcg) == len(stu_ret_docs), "Q4c: Your ndcg should have the same length as there are queries. "
assert stu_ndcg.keys() == stu_ret_docs.keys(), "Q4c: Your ndcg contains invalid queries. "

# NDCG must be between 0 and 1
for q_id in stu_ndcg:
    assert 0.0 <= stu_ndcg[q_id] <= 1.0, f"Q4c: Your NDCG for {q_id} should be between 0 and 1. "

# Check it is still correct even if n exceeds max_docs
stu_ndcg_max = calc_NDCG_at_n(stu_ret_docs, stu_reljudges, n=(max_docs + 2), base=base)

assert all(map(math.isclose, stu_ndcg.values(), stu_ndcg_max.values())), "Q4c: When n exceeds max_docs, your function should return the same results as when n=-1. "

# Some hidden tests

del stu_inv_index, stu_queries, stu_reljudges, stu_ret_docs
del stu_ndcg, stu_ndcg_max
del min_df, max_docs, base